In [1]:
cd ~/Dropbox/project_4

/home/ubuntu/Dropbox/project_4


In [4]:
from hwmf import utils, data
import re
import gensim
from gensim.models import doc2vec

In [3]:
name_to_bio_lemmas = utils.load_data('character_bios_lemmatized.pickle')
name_to_bio_tokens = utils.load_data('character_bios_tokenized.pickle')

In [5]:
name_to_article = data.get_character_articles()

In [25]:
name_to_article['Bender (Futurama)']

'Bender Bending Rodríguez (designated in-universe as Bending Unit 22, unit number 1,729, serial number 2716057) is a fictional character who is one of the main characters in the animated television series Futurama. He was created by series creators Matt Groening and David X. Cohen and is voiced by John DiMaggio. He fulfills a comic, antihero-type role in Futurama and is described by fellow character Leela as an "alcoholic, whore-mongering, chain-smoking gambler".\n\nAccording to the character\'s backstory, Bender was built in Tijuana, Mexico (the other characters refer to his "swarthy Latin charm"). Viewers are informed, through his own testimony, of Bender\'s prejudice against non-robots. For example, one of his signature expressions is "kill all humans". Exceptions who are not subject to Bender\'s prejudicial attitude are those individuals on his "Do Not Kill" list, which seems to comprise only his best friend Philip J. Fry and his colleague Hermes Conrad (Hermes is added after the e

In [ ]:
[(key, value) for key, value in name_to_bio_tokens.items() if key in name_to_article]

In [7]:
name_to_bio_lemmas = dict((key, value) for key, value in name_to_bio_tokens.items() if key in name_to_article) 

NameError: name 'name_to_bio_tokens' is not defined

In [126]:
gensim.utils.simple_preprocess("that's how we do it!\nyeah man")

['that', 'how', 'we', 'do', 'it', 'yeah', 'man']

In [26]:
len(name_to_article)

14271

In [28]:
name_to_article_processed['Luke Skywalker']

TaggedDocument(words=[], tags=['Luke Skywalker'])

In [31]:
title_black_list = utils.load_values('character_article_title_black_list.csv')
word_black_list = utils.load_values('character_article_word_black_list.csv')

def remove_black_listed(doc_string):
    black_list = sorted(word_black_list, key=lambda x: -len(x))
    patterns = []
    for pattern in word_black_list:
        patterns.append(r'\b' + pattern + '\b')
    patterns += word_black_list
    for pattern in patterns:
        doc_string = re.sub(pattern, '', doc_string)
    return doc_string

def to_word_list(sents):
    merged = merge_sents(sents)
    merged = remove_black_listed(merged)
    return gensim.utils.simple_preprocess(merged)

train_corpus = []
name_to_article_processed = {}
count = 0
for name, article in name_to_article.items():
    processed = remove_black_listed(article)
    processed = gensim.utils.simple_preprocess(processed)
    doc = doc2vec.TaggedDocument(processed, [name])
    train_corpus.append(doc)
    name_to_article_processed[name] = processed
    count += 1
    if count % 1000 == 0:
        print(count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [21]:
len(train_corpus)

14271

In [113]:
to_word_list(name_to_article['Bender (Futurama)'])

['accord',
 'back',
 'build',
 'high',
 'tech',
 'industrial',
 'metalworking',
 'robot',
 'build',
 'manufacturing',
 'facility',
 'unlike',
 'robot',
 'mortal',
 'accord',
 'calculation',
 'year',
 'live',
 'after',
 'report',
 'defect',
 'manufacturer',
 'barely',
 'escape',
 'death',
 'guide',
 'missile',
 'robot',
 'death',
 'squad',
 'dispatch',
 'mom',
 'order',
 'eliminate',
 'effectively',
 'defective',
 'product',
 'market',
 'job',
 'metalworking',
 'fy',
 'bend',
 'steel',
 'girder',
 'construction',
 'suicide',
 'booth',
 'apartment',
 'although',
 'pair',
 'enjoy',
 'live',
 'portray',
 'manipulate',
 'guileless',
 'friend',
 'in',
 'early',
 'show',
 'prefer',
 'occupy',
 'small',
 'area',
 'apartment',
 'like',
 'closet',
 'refer',
 'cozy',
 'later',
 'show',
 'individual',
 'bedroom',
 'like',
 'hat',
 'magnet',
 'near',
 'pathological',
 'fear',
 'electric',
 'opener',
 'magnet',
 'because',
 'uncontrollably',
 'start',
 'sing',
 'folk',
 'song',
 'near',
 'head',
 'm

In [11]:
class DocIterator(object):
    
    def __init__(self, doc_list):
        self.doc_list = doc_list

    def __iter__(self):
        for doc in self.doc_list:
            # print 'creating tagged document...%d' % idx
            yield doc
            
corpus = DocIterator(train_corpus)

In [44]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [35]:
# model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=50)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, window=10, workers=6, epochs=50)


In [24]:
train_corpus[0]

TaggedDocument(words=[], tags=['Achilles'])

In [34]:

# = gensim.utils.grouper(train_corpus, 4)
# iter_vals = gensim.models.word2vec.itervalues(name_to_article_processed)

# %time model.train(train_corpus, total_examples=model.corpus_count, epochs=5000)

model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=5000)

KeyboardInterrupt: 

In [159]:
gensim.models.doc2vec.FAST_VERSION

1

In [158]:
model.save('first_that_doesnt_suck.model')

In [155]:
# After training
model.docvecs.most_similar('Zuko')

[('Aragorn', 0.6223043203353882),
 ('Temugin', 0.6067795157432556),
 ('Boromir', 0.6062975525856018),
 ('Galadriel', 0.5872621536254883),
 ('Yoda', 0.5668113827705383),
 ('William Bush (Hornblower)', 0.561889111995697),
 ('Kratos (God of War)', 0.5594049692153931),
 ('Jim Raynor', 0.5549412965774536),
 ('Théoden', 0.5423216223716736),
 ('Forerunner (DC Comics)', 0.5411306619644165)]

In [136]:
model.docvecs.most_similar('Zuko')

[('Aragorn', 0.6809361577033997),
 ('Aihal', 0.5727719068527222),
 ('Ben Linus', 0.5720242857933044),
 ('Boromir', 0.560214638710022),
 ('Viserys Targaryen', 0.5582283735275269),
 ('Cardinal Lamberto', 0.5393531918525696),
 ('Count Dooku', 0.5358583331108093),
 ('Korra', 0.5352246761322021),
 ('Yoda', 0.5285734534263611),
 ('Mulan (Disney character)', 0.5230451226234436)]

In [156]:
# After
model.docvecs.most_similar('Aang')

[('Bran Stark', 0.6570450067520142),
 ('Naruto Uzumaki', 0.6490170955657959),
 ('Aerith Gainsborough', 0.6444741487503052),
 ('Maximilian Jenius', 0.6083365082740784),
 ('Kakashi Hatake', 0.5698568820953369),
 ('Goku', 0.5684899091720581),
 ('Lelouch Lamperouge', 0.563614010810852),
 ('Paul Atreides', 0.5528197288513184),
 ('Boruto Uzumaki', 0.5497936606407166),
 ('Momiji (Ninja Gaiden)', 0.5424197316169739)]

In [137]:
model.docvecs.most_similar('Aang')

[('Aerith Gainsborough', 0.6787601709365845),
 ('Bran Stark', 0.5986438393592834),
 ('Goku', 0.5943489670753479),
 ('Gohan', 0.5703436136245728),
 ('Anna (Frozen)', 0.5645865201950073),
 ('Lelouch Lamperouge', 0.5546486377716064),
 ('Count Dooku', 0.5480912923812866),
 ('Paul Atreides', 0.545677900314331),
 ('Kyle Reese', 0.5450496673583984),
 ('Korra', 0.5447707176208496)]

In [139]:
model.docvecs.most_similar('Yoda')

[('Qui-Gon Jinn', 0.7101578712463379),
 ('Luke Skywalker', 0.6712669730186462),
 ('Optimus Prime', 0.6421325206756592),
 ('Vegeta', 0.6367337703704834),
 ('Hao Asakura', 0.6160750389099121),
 ('Obi-Wan Kenobi', 0.6145543456077576),
 ('Raoh', 0.5810256004333496),
 ('Darth Bane', 0.5740638971328735),
 ('Sephiroth (Final Fantasy)', 0.5709008574485779),
 ('Aqua (Kingdom Hearts)', 0.5666394829750061)]

In [157]:
# After
model.docvecs.most_similar('Yoda')

[('Qui-Gon Jinn', 0.6633567810058594),
 ('Luke Skywalker', 0.6167474985122681),
 ('Optimus Prime', 0.6107190847396851),
 ('Scourge (Transformers)', 0.5825558304786682),
 ('Kanan Jarrus', 0.5685822367668152),
 ('Zuko', 0.5668113231658936),
 ('Gimli (Middle-earth)', 0.565727710723877),
 ('Mokujin', 0.5602508783340454),
 ('Albus Dumbledore', 0.5537698268890381),
 ('Beorn', 0.5510784387588501)]

In [68]:
def find_most_similar(word_list):
    inferred_vector = model.infer_vector(word_list)
#     print(len(inferred_vector))
    sims = model.docvecs.most_similar([inferred_vector], topn=10)
    return sims

In [69]:
word_list = to_word_list(name_to_article['Bender (Futurama)'])
find_most_similar(word_list)

[('Bender (Futurama)', 0.9172766208648682),
 ('Burt Gummer', 0.6373168230056763),
 ('Sammy the Shunter', 0.6062910556793213),
 ('The Smurfs (comics)', 0.5927327275276184),
 ('Karla (character)', 0.5844019055366516),
 ('Nick Carter-Killmaster', 0.5783843398094177),
 ('Mega Mindy', 0.5779253840446472),
 ('Yig', 0.5778729915618896),
 ('Jerom', 0.5753814578056335),
 ('Pluk van de Petteflet', 0.5753750801086426)]

In [18]:
for doc in train_corpus_docs:
    page_name = doc.tags[0]
    inferred_vector = model.infer_vector(train_corpus[page_name].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=5
    print('Character:', page_name, 'Most Similar:'
    rank = [docid for docid, sim in sims].index(page_name)
    print(rank)
    break

0
